In [2]:
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.4 MB/s eta 0:00:00


In [14]:
import pandas as pd
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk

# STOP WORD REMOVER
stop_word = StopWordRemoverFactory().create_stop_word_remover()
# STEMMER
stemmer = StemmerFactory().create_stemmer()

remove_tanda_baca = nltk.RegexpTokenizer(r"\w+")

# Fungsi untuk menghapus stopwords dari teks


def pre_process_text(text):
    text = ' '.join(remove_tanda_baca.tokenize(text))
    return stop_word.remove(stemmer.stem(text))

# Fungsi untuk mengganti nilai rating
def convert_rating(rating):
    if rating > 3:
        return 1
    elif rating < 3:
        return 0
    else:
        return 0


df = pd.read_csv('/content/reviews_mandiri.csv')

rating_positif = df[df['rating'] > 3].sample(n=2000, axis=0)
rating_negatif = df[df['rating'] < 3].sample(n=2000, axis=0)

slice_df = pd.concat([rating_positif, rating_negatif])
slice_df['is_positive'] = slice_df['rating'].apply(convert_rating)
slice_df['review'] = slice_df['review'].apply(pre_process_text)

In [31]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

attribute = slice_df['review'].values
label = slice_df['is_positive'].values

data_latih, data_test, label_latih, label_test = train_test_split(attribute,label,test_size=0.2)
tokenizer = Tokenizer(num_words=20000,oov_token='x')
tokenizer.fit_on_texts(data_latih)

sekuens_latih = tokenizer.texts_to_sequences(data_latih)
sekuens_test = tokenizer.texts_to_sequences(data_test)

pad_latih = pad_sequences(sekuens_latih,padding='post',maxlen=200)
pad_test = pad_sequences(sekuens_test,padding='post',maxlen=200)

In [35]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Contoh model dasar
model_base = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=15000, output_dim=1, input_length=200),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Model dasar dengan penyetelan hyperparameter
model_base.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

ACCURACY_TRESHOLD = 90e-2
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = None):
        accuracy = logs.get('accuracy')
        if accuracy >= ACCURACY_TRESHOLD:
            print(f'\n Epoch {epoch}\n Accuracy has reach = {logs["accuracy"]*100:.2f}%/n training has been stopped.')
            self.model.stop_training = True

model_base.fit(
    pad_latih,
    label_latih,
    epochs=50,
    validation_data=(pad_test, label_test),
    callbacks=[MyCallback()]
)

Epoch 1/50
100/100 [==============================] - 14s 124ms/step - loss: 0.5905 - accuracy: 0.7206 - val_loss: 0.3311 - val_accuracy: 0.8750
Epoch 2/50
100/100 [==============================] - 11s 107ms/step - loss: 0.3045 - accuracy: 0.8978 - val_loss: 0.4768 - val_accuracy: 0.8350
Epoch 3/50
100/100 [==============================] - ETA: 0s - loss: 0.2329 - accuracy: 0.9237
 Epoch 2
 Accuracy has reach = 92.37%/n training has been stopped.
100/100 [==============================] - 12s 125ms/step - loss: 0.2329 - accuracy: 0.9237 - val_loss: 0.3445 - val_accuracy: 0.8575


In [38]:
model_base.save('model_sigmoid.keras')